# Validation against CHES and POPPA Expert Ratings

In [74]:
import pandas as pd

import src

## CHES Data

In [75]:
ches = pd.read_csv(src.PATH / "data/CHES/CHES2019V3.csv")

ches = ches.loc[(ches.country == 3), ["party", "people_vs_elite", "antielite_salience"]]

ches["party"] = ches["party"].replace({"GRUNEN": "Grüne", "LINKE": "DIE LINKE."})

ches = ches.rename({"antielite_salience": "elite", "people_vs_elite": "pplcentr"}, axis=1)

seats_cdu = 200
seats_csu = 45

row = pd.Series({"party": "CDU/CSU"})
for var in ["pplcentr", "elite"]:
    row[var] = (
        (ches.loc[ches.party == "CSU", var].values[0] * seats_csu)
        + ((ches.loc[ches.party == "CDU", var].values[0] * seats_cdu))
    ) / (seats_cdu + seats_csu)

ches = pd.concat([ches.T, row], axis=1).T

ches["pplcentr"] = ches["pplcentr"].astype(float)
ches["elite"] = ches["elite"].astype(float)

In [76]:
ches

,party,pplcentr,elite
21,CDU,1.700000,0.750000
22,SPD,2.450000,1.350000
23,FDP,2.350000,1.250000
24,Grüne,4.900000,1.800000
25,DIE LINKE.,4.350000,4.450000
26,CSU,2.842105,1.550000
27,AfD,8.315789,9.700000
28,Piraten,8.900000,8.800000
29,DieTier,9.666667,10.000000
0,CDU/CSU,1.909774,0.896939


In [77]:
df_ours = pd.read_csv(src.PATH / "data/figure_1_numbers.csv")
df_ours = df_ours[df_ours.electoral_term == 19]
df_ours["variable"] = df_ours["variable"].replace(
    {"(a) Anti-Elitism": "elite", "(b) People-Centrism": "pplcentr"}
)
df_ours = df_ours[df_ours.variable.isin(["elite", "pplcentr"])]
df_ours = df_ours.pivot(index="Party", columns="variable", values="value").reset_index()

In [78]:
df = pd.merge(
    ches, df_ours, how="inner", left_on="party", right_on="Party", suffixes=["_ches", ""])

## POPPA Data


Source: 
Meijers, Maurits; Zaslove, Andrej, 2020, "Populism and Political Parties Expert Survey 2018 (POPPA)", [https://doi.org/10.7910/DVN/8NEL7B](https://doi.org/10.7910/DVN/8NEL7B)

In [79]:
poppa = pd.read_csv(src.PATH / "data/POPPA/party_means.tab",
                    sep='\t')
poppa = poppa.loc[(poppa.country_id == 12), ["party", "peoplecentrism", "antielitism"]]
poppa["party"] = poppa["party"].replace({"B90Grune": "Grüne", "Linke": "DIE LINKE."})
poppa = poppa.rename({"antielitism": "elite", "peoplecentrism": "pplcentr"}, axis=1)

In [80]:
row = pd.Series({"party": "CDU/CSU"})
for var in ["pplcentr", "elite"]:
    row[var] = (
        (poppa.loc[poppa.party == "CSU", var].values[0] * seats_csu)
        + ((poppa.loc[poppa.party == "CDU", var].values[0] * seats_cdu))
    ) / (seats_cdu + seats_csu)

poppa = pd.concat([poppa.T, row], axis=1).T

poppa["pplcentr"] = poppa["pplcentr"].astype(float)
poppa["elite"] = poppa["elite"].astype(float)

In [81]:
df = pd.merge(
    poppa, df, how="inner", left_on="party", right_on="Party", suffixes=["_poppa", ""]
)

## Correlations

In [82]:
df = df.drop("Party", axis=1)
df.corr(method="pearson", numeric_only=True)

,pplcentr_poppa,elite_poppa,pplcentr_ches,elite_ches,elite,pplcentr
pplcentr_poppa,1.000000,0.978300,0.981106,0.983208,0.914437,0.822762
elite_poppa,0.978300,1.000000,0.964664,0.950454,0.830183,0.904655
pplcentr_ches,0.981106,0.964664,1.000000,0.931613,0.934325,0.785076
elite_ches,0.983208,0.950454,0.931613,1.000000,0.871626,0.797684
elite,0.914437,0.830183,0.934325,0.871626,1.000000,0.591293
pplcentr,0.822762,0.904655,0.785076,0.797684,0.591293,1.000000
